<a href="https://colab.research.google.com/github/cygkichi/bucketbrigade/blob/master/smartmill_ver04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img height="60px" src="/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

## スマートミル(Smart Mill)へようこそ

Smart Mill はコーヒーミル内の状態を音声から推測します。

## セットアップ

### 音声データのインポート

In [10]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/smartmill/dryer_sample

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cold_01_01.wav	cold_01_16.wav	cold_02_09.wav	 turbo_01_14.wav
cold_01_02.wav	cold_01_17.wav	cold_02_10.wav	 turbo_01_15.wav
cold_01_03.wav	cold_01_18.wav	turbo_01_01.wav  turbo_01_16.wav
cold_01_04.wav	cold_01_19.wav	turbo_01_02.wav  turbo_02_01.wav
cold_01_05.wav	cold_01_20.wav	turbo_01_03.wav  turbo_02_02.wav
cold_01_06.wav	cold_01_21.wav	turbo_01_04.wav  turbo_02_03.wav
cold_01_07.wav	cold_01_22.wav	turbo_01_05.wav  turbo_02_04.wav
cold_01_08.wav	cold_02_01.wav	turbo_01_06.wav  turbo_02_05.wav
cold_01_09.wav	cold_02_02.wav	turbo_01_07.wav  turbo_02_06.wav
cold_01_10.wav	cold_02_03.wav	turbo_01_08.wav  turbo_02_07.wav
cold_01_11.wav	cold_02_04.wav	turbo_01_09.wav  turbo_02_08.wav
cold_01_12.wav	cold_02_05.wav	turbo_01_10.wav  turbo_02_09.wav
cold_01_13.wav	cold_02_06.wav	turbo_01_11.wav
cold_01_14.wav	cold_02_07.wav	turbo_01_12.wav
cold_01_15.wav	cold_02_08

### 必要ライブラリのインポート

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.signal
from scipy.io import wavfile
import glob
import os
import librosa
from sklearn.model_selection import train_test_split
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)
from keras.models import Model
from keras.utils import Sequence, to_categorical

### クラス・関数のインポート

### Config
  * nperseg：窓幅(STFT)
  * hop_size：窓のずらし幅(STFT)
  * freq_range：周波数[Hz]
  * fs：サンプリング周波数[Hz]


In [0]:
class Config(object):
    def __init__(self, nperseg=2**13, hop_size=2**7,
                freq_range=[0,4000], fs = 44100,
                n_class=2, input_length=100):
        self.fs         = fs
        self.nperseg    = nperseg     
        self.hop_size   = hop_size
        self.noverlap   = nperseg - hop_size
        self.freq_range = np.array(freq_range)
        
        self.n_class      = n_class
        self.input_length = input_length

### make_Image 関数
  * filename：wavファイルのパス
  * config：Configのインスタンス
  * pngname：保存画像名 Noneの場合表示のみ

In [0]:
def draw_image(filename, config, pngname=None):
    fs, data = wavfile.read(filename)
    data = data[:,0] / 32768
    
    fig = plt.figure(figsize=(3, 4))
    ax_raw  = plt.axes([0.1, 0.8 , 0.8, 0.15])    
    ax_stft = plt.axes([0.1, 0.45, 0.8, 0.25])    
    ax_fft  = plt.axes([0.1, 0.1 , 0.8, 0.25])    
    
    title_text = filename.split('/')[-1] 
    
    ax_raw.set_title(title_text)
    ax_raw.plot(data)
    
    f, t, zxx = scipy.signal.stft(data, nperseg=config.nperseg, \
                                  noverlap=config.noverlap, fs=config.fs)
    power = np.log10(np.abs(zxx))
    ax_stft.set_ylim(config.freq_range)
    ax_stft.pcolormesh(t, f, power, cmap='jet', vmin=-4.5, vmax=-3)

    fft = np.average(power, axis=1)
    ax_fft.set_xlim(config.freq_range)
    ax_fft.set_ylim(-4.5,-3)
    ax_fft.plot(f,fft)
    
    if pngname is None:
        plt.show()
    else:
        plt.savefig(pngname)
    

### モデル取得関数

In [0]:
def get_densely_connected_model(config):
    '''
    ref : https://keras.io/ja/getting-started/functional-api-guide/
    例1.全結合ネットワーク
    '''
    nclass = config.n_class
    input_length = config.input_length
    
    inputs = Input(shape=(input_length,))
    
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(nclass, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Run Smart Mill

### データセットの可視化

In [0]:
if __name__ == '__main__':
    config = Config()
    dirname = '/content/drive/My Drive/smartmill/dryer_sample/'
    filenames = glob.glob(dirname + '*.wav')
    for f in filenames[::10]:
        '''10は間引き
        '''
        pngname = f.split('/')[-1].replace('.wav', '.png')
        draw_image(f, config, pngname)

### 学習&評価

In [46]:
if __name__ == '__main__':
    config = Config()
    dirname = '/content/drive/My Drive/smartmill/dryer_sample/'
    filenames = glob.glob(dirname + '*.wav')

    datas  = []
    labels = []
    tag2label = {'cold'  : np.array([1.0, 0.0]),
                 'turbo' : np.array([0.0, 1.0])}
    for filename in filenames:
        for i in range(100):
            fs, data = wavfile.read(filename)
            data = data[:,0] / 32768
            data = data[100*i:100*i+100]
            datas.append(data)

            tag = filename.split('/')[-1].split('_')[0]
            label = tag2label[tag]
            labels.append(label)
    
    datas  = np.array(datas)
    labels = np.array(labels)
    
    model = get_densely_connected_model(config)
    
    X_train, X_test, y_train, y_test = train_test_split(
        datas, labels, test_size=0.33, random_state=42)
    
    model.fit(X_train, y_train, epochs=10, batch_size=5)
    
    loss_and_metric = model.evaluate(X_test, y_test, batch_size=5)
    print(loss_and_metric)

Epoch 1/10
3819/3819 [==============================] - 5s 1ms/step - loss: 0.3440 - acc: 0.9068
Epoch 2/10
3819/3819 [==============================] - 4s 966us/step - loss: 0.0447 - acc: 0.9914
Epoch 3/10
3819/3819 [==============================] - 4s 964us/step - loss: 0.0128 - acc: 0.9979
Epoch 4/10
3819/3819 [==============================] - 4s 923us/step - loss: 0.0055 - acc: 0.9990
Epoch 5/10
3819/3819 [==============================] - 3s 855us/step - loss: 0.0022 - acc: 0.9995
Epoch 6/10
3819/3819 [==============================] - 3s 842us/step - loss: 5.2347e-04 - acc: 1.0000
Epoch 7/10
3819/3819 [==============================] - 3s 838us/step - loss: 1.3825e-04 - acc: 1.0000
Epoch 8/10
3819/3819 [==============================] - 3s 843us/step - loss: 3.8147e-05 - acc: 1.0000
Epoch 9/10
3819/3819 [==============================] - 3s 875us/step - loss: 9.1611e-05 - acc: 1.0000
Epoch 10/10
1881/1881 [==============================] - 1s 546us/step
[0.024070204128510297, 0